# 掛載 Google 雲端硬碟

In [ ]:
#@title <font size="5">Mount Google Drive</font> { vertical-output: true }
from google.colab import drive
drive.mount('/content/drive')

# 安裝套件

In [ ]:
#@title <font size="5">安裝 NVIDIA 的 CUDA-Toolkit ([參考資料](https://github.com/camenduru/gaussian-splatting-colab/issues))</font> { vertical-output: true }
!sudo apt install nvidia-cuda-toolkit

In [ ]:
#@title <font size="5">下載 gaussian-splatting</font> { vertical-output: true }
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

In [ ]:
#@title <font size="5">安裝 diff_gaussian_rasterization 和 simple_knn ([參考資料](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_colab.ipynb))</font> { vertical-output: true }
%cd /content/gaussian-splatting
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl

In [ ]:
#@title <font size="5">安裝 colmap ([參考資料](https://colab.research.google.com/drive/13u8DX9BNzQwiyPPCB7_4DbSxiQ5-_nGF#scrollTo=ahZzbkIbSQAL))</font> { vertical-output: true }
%cd /content
!apt-get install \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev
!apt-get install xvfb
# libglvnd is needed for COLMAP to run on Google Colab (https://github.com/colmap/colmap/issues/1271#issuecomment-931900582)
!git clone https://github.com/NVIDIA/libglvnd
%cd /content/libglvnd
!apt-get install libxext-dev libx11-dev x11proto-gl-dev
!apt-get install autoconf automake libtool
!apt-get install libffi-dev
!./autogen.sh
!./configure
!make -j4
!make install
# Download and extract the pre-compiled COLMAP library
%cd /content
!gdown 1PyyNKY2mt4dHlYN5WcPnvWdV2nufQru1
!tar -C /usr -zxf colmap-3.8.tar.gz
# Install other Python libraries for the data preparation scripts
!pip install \
    addict \
    k3d \
    opencv-python-headless \
    pillow \
    plotly \
    pyyaml \
    trimesh

In [ ]:
#@title <font size="5">安裝 FFmpeg ([參考資料](https://colab.research.google.com/github/google/nerfies/blob/main/notebooks/Nerfies_Capture_Processing.ipynb))</font> { vertical-output: true }
!apt-get install ffmpeg

# 運行 FFmpeg

In [ ]:
#@title <font size="5">Run FFmpeg</font> { vertical-output: true }
import os
src_video = "/content/drive/MyDrive/\u003Cvideo_path>" #@param {type:"string"}
fps = "2" #@param {type:"string"}
dst_folder = "/content/gaussian-splatting/data/\u003Cdata_name>/input" #@param {type:"string"}

if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

os.chdir(dst_folder)

!ffmpeg -i {src_video} -qscale:v 1 -qmin 1 -vf fps={fps} %04d.jpg

# 運行 3D_gaussian_splatting

In [ ]:
#@title <font size="5">複製照片來源(自選)</font> { vertical-output: true }
import shutil
import os

# 設定來源和目標資料夾
src_folder = "/content/drive/MyDrive/\u003Cphoto_path>" #@param {type:"string"}
dst_folder = "/content/gaussian-splatting/data/\u003Cfile_name>/input" #@param {type:"string"}

# 複製整個資料夾
shutil.copytree(src_folder, dst_folder)

In [ ]:
#@title <font size="5">執行 convert.py</font> { vertical-output: true }
Folder = "/content/gaussian-splatting/data/\u003Cfile_name>" #@param {type:"string"}

%cd /content/gaussian-splatting
!python convert.py -s {Folder}

In [ ]:
#@title <font size="5">複製 Sourcedata 到雲端硬碟(備份用)</font> { vertical-output: true }
import shutil
import os

# 設定來源和目標資料夾
src_folder = "/content/gaussian-splatting/data/\u003Cfile_name>" #@param {type:"string"}
dst_folder = "/content/drive/MyDrive/Sourcedata/\u003Cfile_name>" #@param {type:"string"}

# 複製整個資料夾
shutil.copytree(src_folder, dst_folder)

In [ ]:
#@title <font size="5">執行 train.py</font> { vertical-output: true }
Folder = "/content/gaussian-splatting/data/\u003Cfile_name>" #@param {type:"string"}

%cd /content/gaussian-splatting
!python train.py -s {Folder}

In [ ]:
#@title <font size="5">複製 Output 到雲端硬碟(備份用)</font> { vertical-output: true }
import shutil
import os

# 設定來源和目標資料夾
src_folder = "/content/gaussian-splatting/output" #@param {type:"string"}
dst_folder = "/content/drive/MyDrive/Output/\u003Cfile_name>" #@param {type:"string"}

# 複製整個資料夾
shutil.copytree(src_folder, dst_folder)

# 實驗功能

In [ ]:
#@title <font size="5">輸入影片路徑一次運行全部</font> { vertical-output: true }

# @markdown ### 輸入影片路徑
video_path = "/content/drive/MyDrive/3D_gaussian_splatting/\u003Cvideo_path>.mp4" #@param {type:"string"}
fps = "1" #@param {type:"string"}
filename = os.path.splitext(os.path.basename(video_path))[0]

dst_folder = f"/content/gaussian-splatting/data/{filename}/input"
train_data = f"/content/gaussian-splatting/data/{filename}"
dst_sourcedata = f"/content/drive/MyDrive/SourceData/{filename}"
dst_output = f"/content/drive/MyDrive/Output/{filename}"

import os
import shutil

# video to photo
if not os.path.exists(dst_folder):
  os.makedirs(dst_folder)
os.chdir(dst_folder)
!ffmpeg -i {video_path} -qscale:v 1 -qmin 1 -vf fps={fps} %04d.jpg

# convert
%cd /content/gaussian-splatting
!python convert.py -s {train_data}

# downloade source_data
src_sourcedata = f"/content/gaussian-splatting/data/{filename}"
if os.path.exists(dst_sourcedata):
    shutil.rmtree(dst_sourcedata)
shutil.copytree(src_sourcedata, dst_sourcedata)

# train
!python train.py -s {train_data}

# downloade output
src_output = "/content/gaussian-splatting/output"
if os.path.exists(dst_output):
    shutil.rmtree(dst_output)
shutil.copytree(src_output, dst_output)